In [1]:
from pyLAMMPS import LAMMPS_setup

In [2]:
lammps_setup = LAMMPS_setup( system_setup = "test/input/setup_eoh_wat.yaml", 
                             simulation_default = "test/input/defaults.yaml",
                             simulation_ensemble = "test/input/ensemble.yaml",
                             simulation_sampling = "test/input/sampling.yaml",
                             submission_command = "sbatch"
                            )

In [6]:
folder_name = "water_pure"
solute = "ethanol"
combined_lambdas = [ 0.0, 0.1, 0.2, 0.25, 0.275, 0.3, 0.325, 0.35, 0.4, 
                     0.5, 0.7, 0.9, 1.0, 1.1, 1.3, 1.5, 1.7, 1.9, 2.0 
                    ]

coupling_settings_file = "test/input/coupling_settings.yaml"
ensembles = [ "em", "npt", "npt" ]
simulation_times = [ 0.0, 8.0, 10.0 ]

initial_systems = []

lammps_setup.prepare_coupling_simulation( folder_name = folder_name, solute = solute, combined_lambdas = combined_lambdas,
                                          coupling_settings_file = coupling_settings_file, ensembles = ensembles, 
                                          simulation_times = simulation_times, initial_systems = initial_systems
                                        )

KeyError: 'Molecules could not be matched!\n'

In [ ]:
## FIX COORDINATE problem. in get_molecule_coordinates

In [9]:
lammps_setup.submit_simulation()


Submitting simulations at Temperature = 90 K, Pressure = 8 bar

Submitting job: /home/st/st_us-041610/st_ac137577/software/vle_using_solvation_free_energy/test/lj1_pure/test1/temp_89.9_pres_7.8/copy_0/lambda_0/job_90_8.sh
Submitted batch job 23414842


Submitting job: /home/st/st_us-041610/st_ac137577/software/vle_using_solvation_free_energy/test/lj1_pure/test1/temp_89.9_pres_7.8/copy_0/lambda_1/job_90_8.sh
Submitted batch job 23414843


Submitting job: /home/st/st_us-041610/st_ac137577/software/vle_using_solvation_free_energy/test/lj1_pure/test1/temp_89.9_pres_7.8/copy_0/lambda_2/job_90_8.sh
Submitted batch job 23414844


Submitting job: /home/st/st_us-041610/st_ac137577/software/vle_using_solvation_free_energy/test/lj1_pure/test1/temp_89.9_pres_7.8/copy_0/lambda_3/job_90_8.sh
Submitted batch job 23414845


Submitting job: /home/st/st_us-041610/st_ac137577/software/vle_using_solvation_free_energy/test/lj1_pure/test1/temp_89.9_pres_7.8/copy_0/lambda_4/job_90_8.sh
Submitted batch job 2

In [3]:
import re
import glob
from itertools import groupby
from pyLAMMPS.analysis.solvation_free_energy import get_free_energy_difference, visualize_dudl

def analysis_free_energy( self, analysis_folder: str, solute: str, ensemble: str, 
                          method: str="MBAR", fraction: float=0.0, 
                          decorrelate: bool=True, visualize: bool=False  ):
    """
    Extracts free energy difference for a specified folder and solute and ensemble.

    Parameters:
     - analysis_folder (str): The name of the folder where the analysis will be performed.
     - solute (str): Solute under investigation
     - ensemble (str): The name of the ensemble for which properties will be extracted. Should be xx_ensemble.
     - method (str, optional): The free energy method that should be used. Defaults to "MBAR".
     - fraction (float, optional): The fraction of data to be discarded from the beginning of the simulation. Defaults to 0.0.
     - decorrelate (bool, optional): Whether to decorrelate the data before estimating the free energy difference. Defaults to True.

    Returns:
        None

    The method searches for output files in the specified analysis folder that match the given ensemble.
    For each group of files with the same temperature and pressure, the properties are extracted using alchempy.
    The extracted properties are then averaged over all copies and the mean and standard deviation are calculated.
    The averaged values and the extracted data for each copy are saved as a JSON file in the destination folder.
    The extracted values are also added to the class's analysis dictionary.
    """
    
    # Define folder for analysis
    sim_folder = f'{self.system_setup["folder"]}/{self.system_setup["name"]}/{analysis_folder}'#/{solute}'

    # Seperatre the ensemble name to determine output files
    ensemble_name = "_".join(ensemble.split("_")[1:])
    
    print(f"\nExtract solvation free energy results for solute: {solute}\n")

    # Loop over each temperature & pressure state
    for temperature, pressure in zip( self.system_setup["temperature"], self.system_setup["pressure"] ):
        
        # Define folder for specific temp and pressure state
        state_folder = f"{sim_folder}/temp_{temperature:.1f}_pres_{pressure:.1f}"

        # Search for available copies
        files = glob.glob( f"{state_folder}/copy_*/lambda_*/{ensemble}/{ensemble_name}.fep" )
        files.sort( key=lambda x: ( int(re.search(r'copy_(\d+)', x).group(1)), 
                                    int(re.search(r'lambda_(\d+)', x).group(1))
                                  )
                  )


        if len(files) == 0:
            raise KeyError(f"No files found machting the ensemble: {ensemble} in folder\n:   {state_folder}")

        print(f"Temperature: {temperature}, Pressure: {pressure}\n   "+"\n   ".join(files) + "\n")
        
        # Sort in copies 
        data_list = [ get_free_energy_difference(list(group), temperature, method, fraction, decorrelate) for 
                      _,group in groupby( files, key=lambda x: int(re.search(r'copy_(\d+)', x).group(1)) ) 
                    ]

        # Visualize dH/dl plots if wanted
        if method in ["TI", "TI_spline"] and visualize:
            for copy,group in groupby( files, key=lambda x: int(re.search(r'copy_(\d+)', x).group(1)) ):
                visualize_dudl( fep_files = group, T = temperature, 
                                fraction = fraction, decorrelate = decorrelate,
                                save_path = f"{state_folder}/copy_{copy}"  
                              )

        if len(data_list) == 0:
            raise KeyError("No data was extracted!")
        
        print(data_list)

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

********* JAX NOT FOUND *********
 PyMBAR can run faster with JAX  
 But will work fine without it   
Either install with pip or conda:
      pip install pybar[jax]     
               OR                
      conda install pymbar       
*********************************


In [9]:
analysis_free_energy( lammps_setup, "test0", "lj1", "01_npt", "TI" )

2024-04-18 15:25:58.790 | DEBUG    | alchemlyb.preprocessing.subsampling:statistical_inefficiency:520 - Running statistical inefficiency analysis.
2024-04-18 15:25:58.792 | DEBUG    | alchemlyb.preprocessing.subsampling:statistical_inefficiency:522 - Statistical inefficiency: 1.19.
2024-04-18 15:25:58.792 | DEBUG    | alchemlyb.preprocessing.subsampling:statistical_inefficiency:528 - Number of uncorrelated samples: 5000.
2024-04-18 15:25:58.818 | DEBUG    | alchemlyb.preprocessing.subsampling:statistical_inefficiency:520 - Running statistical inefficiency analysis.
2024-04-18 15:25:58.819 | DEBUG    | alchemlyb.preprocessing.subsampling:statistical_inefficiency:522 - Statistical inefficiency: 1.59.
2024-04-18 15:25:58.820 | DEBUG    | alchemlyb.preprocessing.subsampling:statistical_inefficiency:528 - Number of uncorrelated samples: 5000.


2024-04-18 15:25:58.845 | DEBUG    | alchemlyb.preprocessing.subsampling:statistical_inefficiency:520 - Running statistical inefficiency analysis.
2024-04-18 15:25:58.846 | DEBUG    | alchemlyb.preprocessing.subsampling:statistical_inefficiency:522 - Statistical inefficiency: 2.40.
2024-04-18 15:25:58.847 | DEBUG    | alchemlyb.preprocessing.subsampling:statistical_inefficiency:528 - Number of uncorrelated samples: 3333.
2024-04-18 15:25:58.873 | DEBUG    | alchemlyb.preprocessing.subsampling:statistical_inefficiency:520 - Running statistical inefficiency analysis.
2024-04-18 15:25:58.874 | DEBUG    | alchemlyb.preprocessing.subsampling:statistical_inefficiency:522 - Statistical inefficiency: 1.38.
2024-04-18 15:25:58.875 | DEBUG    | alchemlyb.preprocessing.subsampling:statistical_inefficiency:528 - Number of uncorrelated samples: 5000.
2024-04-18 15:25:58.899 | DEBUG    | alchemlyb.preprocessing.subsampling:statistical_inefficiency:520 - Running statistical inefficiency analysis.
202


Extract solvation free energy results for solute: lj1

Temperature: 89.88, Pressure: 7.84
   /home/st/st_us-041610/st_ac137577/software/vle_using_solvation_free_energy/test/lj1_pure/test0/temp_89.9_pres_7.8/copy_0/lambda_0/01_npt/npt.fep
   /home/st/st_us-041610/st_ac137577/software/vle_using_solvation_free_energy/test/lj1_pure/test0/temp_89.9_pres_7.8/copy_0/lambda_1/01_npt/npt.fep
   /home/st/st_us-041610/st_ac137577/software/vle_using_solvation_free_energy/test/lj1_pure/test0/temp_89.9_pres_7.8/copy_0/lambda_2/01_npt/npt.fep
   /home/st/st_us-041610/st_ac137577/software/vle_using_solvation_free_energy/test/lj1_pure/test0/temp_89.9_pres_7.8/copy_0/lambda_3/01_npt/npt.fep
   /home/st/st_us-041610/st_ac137577/software/vle_using_solvation_free_energy/test/lj1_pure/test0/temp_89.9_pres_7.8/copy_0/lambda_4/01_npt/npt.fep
   /home/st/st_us-041610/st_ac137577/software/vle_using_solvation_free_energy/test/lj1_pure/test0/temp_89.9_pres_7.8/copy_0/lambda_5/01_npt/npt.fep

[                pro